In [ ]:
from pyspark.sql.functions import *

In [ ]:
filepath = '/FileStore/tables/fire_data/'

In [ ]:
schema1 = "call_no BIGINT,\
    unit_id STRING,\
    incident_no BIGINT,\
    call_type STRING,\
    call_date STRING,\
    watch_date STRING,\
    received_dttm STRING,\
    entry_dttm STRING,\
    dispatch_dttm STRING,\
    response_dttm STRING,\
    onscene_dttm STRING,\
    transport_dttm STRING,\
    hospital_dttm STRING,\
    call_final_disp STRING,\
    available_dttm STRING,\
    address STRING,\
    city STRING,\
    zipcode INT,\
    battalion STRING,\
    station_area STRING,\
    box INT,\
    original_priority SMALLINT,\
    priority SMALLINT,\
    final_priority SMALLINT,\
    ALS_unit BOOLEAN,\
    call_type_group STRING,\
    no_alarms SMALLINT,\
    unit_type STRING,\
    unit_sequence SMALLINT,\
    fire_prevention_dist SMALLINT,\
    supervisor_dist SMALLINT,\
    neighborhoods STRING,\
    rowid STRING,\
    case_location STRING,\
    data_as_of STRING,\
    data_loaded_at STRING,\
    analysis_neighbourhoods SMALLINT"


In [ ]:
fire_data = spark.read\
    .option('header','true')\
        .option('nullValue','NULL')\
            .schema(schema1)\
                .csv(filepath)

In [ ]:
fire_data.printSchema()

root
 |-- call_no: long (nullable = true)
 |-- unit_id: string (nullable = true)
 |-- incident_no: long (nullable = true)
 |-- call_type: string (nullable = true)
 |-- call_date: string (nullable = true)
 |-- watch_date: string (nullable = true)
 |-- received_dttm: string (nullable = true)
 |-- entry_dttm: string (nullable = true)
 |-- dispatch_dttm: string (nullable = true)
 |-- response_dttm: string (nullable = true)
 |-- onscene_dttm: string (nullable = true)
 |-- transport_dttm: string (nullable = true)
 |-- hospital_dttm: string (nullable = true)
 |-- call_final_disp: string (nullable = true)
 |-- available_dttm: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- battalion: string (nullable = true)
 |-- station_area: string (nullable = true)
 |-- box: integer (nullable = true)
 |-- original_priority: short (nullable = true)
 |-- priority: short (nullable = true)
 |-- final_priority: short

In [ ]:
fire_data = fire_data.withColumn('response_ts',to_timestamp(fire_data['response_dttm'],'MM/dd/yyyy hh:mm:ss a'))
fire_data = fire_data.withColumn('received_ts',to_timestamp(fire_data['received_dttm'],'MM/dd/yyyy hh:mm:ss a'))
fire_data = fire_data.withColumn('call_date_ts',to_timestamp(fire_data['call_date'],'MM/dd/yyyy'))

In [ ]:
fire_data.createOrReplaceTempView("v_fire")

In [ ]:
%sql
-- How many distinct types of calls were made to the fire department?
SELECT COUNT(DISTINCT call_type) AS cnt FROM v_fire;

cnt
32


In [ ]:
%sql
-- What are distinct types of calls made to the fire department?
SELECT DISTINCT call_type FROM v_fire;

call_type
Elevator / Escalator Rescue
Aircraft Emergency
Structure Fire / Smoke in Building
Alarms
Odor (Strange / Unknown)
Citizen Assist / Service Call
HazMat
Watercraft in Distress
Explosion
Oil Spill


In [ ]:
%sql
SELECT * FROM v_fire LIMIT 5;

call_no,unit_id,incident_no,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,onscene_dttm,transport_dttm,hospital_dttm,call_final_disp,available_dttm,address,city,zipcode,battalion,station_area,box,original_priority,priority,final_priority,ALS_unit,call_type_group,no_alarms,unit_type,unit_sequence,fire_prevention_dist,supervisor_dist,neighborhoods,rowid,case_location,data_as_of,data_loaded_at,analysis_neighbourhoods
160743491,71,16029460,Medical Incident,03/14/2016,03/14/2016,03/14/2016 07:56:39 PM,03/14/2016 07:58:11 PM,03/14/2016 07:58:23 PM,03/14/2016 07:58:28 PM,03/14/2016 08:07:14 PM,03/14/2016 08:31:30 PM,03/14/2016 08:37:12 PM,Code 2 Transport,03/14/2016 09:15:04 PM,1100 Block of HOWARD ST,San Francisco,94103,B03,01,2314,2,2,2,true,Non Life-threatening,1,MEDIC,1,2,6,South of Market,160743491-71,POINT (-122.411359138357 37.776494480878),null,11/30/2023 09:35:04 AM,34
141581925,AM06,14054458,Medical Incident,06/07/2014,06/07/2014,06/07/2014 02:53:07 PM,06/07/2014 02:53:07 PM,06/07/2014 02:55:06 PM,06/07/2014 02:55:39 PM,06/07/2014 03:00:07 PM,06/07/2014 03:14:44 PM,06/07/2014 03:28:23 PM,Code 2 Transport,06/07/2014 04:18:14 PM,800 Block of ELLIS ST,San Francisco,94109,B02,03,3115,2,2,2,false,Non Life-threatening,1,PRIVATE,1,2,6,Tenderloin,141581925-AM06,POINT (-122.420175524645 37.783872357642),null,11/30/2023 09:35:04 AM,36
2530180,E16,75851,Structure Fire,09/09/2000,09/09/2000,09/09/2000 02:30:59 PM,09/09/2000 02:31:28 PM,09/09/2000 02:31:37 PM,09/09/2000 02:32:55 PM,09/09/2000 02:33:45 PM,null,null,Other,09/09/2000 03:13:03 PM,2900 Block of FILLMORE ST,SF,94123,B04,16,3552,3,3,3,false,null,1,ENGINE,1,4,2,Marina,002530180-E16,POINT (-122.4353274337 37.796632336083),null,11/30/2023 09:35:04 AM,13
50530023,M14,5014228,Medical Incident,02/22/2005,02/21/2005,02/22/2005 01:32:32 AM,02/22/2005 01:35:25 AM,02/22/2005 01:35:35 AM,null,02/22/2005 01:37:31 AM,null,null,Patient Declined Transport,02/22/2005 02:02:07 AM,700 Block of 14TH AVE,SF,94118,B07,31,7155,3,3,3,true,null,1,MEDIC,1,7,1,Outer Richmond,050530023-M14,POINT (-122.472435000241 37.774043615423),null,11/30/2023 09:35:04 AM,29
230393556,65,23019191,Medical Incident,02/08/2023,02/08/2023,02/08/2023 11:51:09 PM,02/08/2023 11:51:09 PM,02/08/2023 11:51:24 PM,02/08/2023 11:51:27 PM,02/09/2023 12:01:37 AM,02/09/2023 12:19:24 AM,02/09/2023 12:33:30 AM,Code 2 Transport,02/09/2023 12:58:06 AM,700 Block of CAPITOL AVE,San Francisco,94112,B09,33,8373,null,2,2,true,Non Life-threatening,1,MEDIC,1,9,11,Oceanview/Merced/Ingleside,230393556-65,POINT (-122.45913706005143 37.717028652276326),null,11/30/2023 09:35:04 AM,24


In [ ]:
%sql
-- Find out all responses for delayed times greater than 5 mins?
SELECT DISTINCT call_final_disp FROM v_fire 
WHERE date_diff(MINUTE,received_ts,response_ts)>5;


call_final_disp
Medical Examiner
No Merit
Patient Declined Transport
Gone on Arrival
Unable to Locate
Cancelled
SFPD
Other
Against Medical Advice
Code 3 Transport


In [ ]:
%sql
-- 4. What were the most common call types?
SELECT call_type FROM v_fire GROUP BY call_type ORDER BY COUNT(call_type) DESC LIMIT 10;


call_type
Medical Incident
Alarms
Structure Fire
Traffic Collision
Other
Citizen Assist / Service Call
Outside Fire
Water Rescue
Gas Leak (Natural and LP Gases)
Vehicle Fire


In [ ]:
%sql
-- 5. What zip codes accounted for the most common calls?
SELECT zipcode, call_type, COUNT(call_type) FROM v_fire GROUP BY zipcode, call_type ORDER BY 3 DESC LIMIT 10;

zipcode,call_type,count(call_type)
94102,Medical Incident,9671
94103,Medical Incident,8909
94109,Medical Incident,5818
94110,Medical Incident,5652
94124,Medical Incident,3356
94112,Medical Incident,3263
94115,Medical Incident,2776
94107,Medical Incident,2357
94122,Medical Incident,2330
94133,Medical Incident,2215


In [ ]:
%sql
-- 6. What San Francisco neighborhoods are in the zip codes 94102 and 94103?
SELECT DISTINCT neighborhoods FROM v_fire WHERE city = "San Francisco" AND zipcode IN (94102,94103);

neighborhoods
Western Addition
Mission Bay
Hayes Valley
Financial District/South Beach
Nob Hill
Mission
Tenderloin
Potrero Hill
Castro/Upper Market
South of Market


In [ ]:
%sql
-- 7. What was the sum of all calls, average, min, and max of the call response times?
SELECT SUM(date_diff(MINUTE,received_ts,response_ts)) sum,
AVG(date_diff(MINUTE,received_ts,response_ts)) avg,
MIN(date_diff(MINUTE,received_ts,response_ts)) min,
MAX(date_diff(MINUTE,received_ts,response_ts)) max 
FROM v_fire;


sum,avg,min,max
325101,3.489368781461645,0,1375


In [ ]:
%sql
-- 8. How many distinct years of data are in the CSV le?

SELECT COUNT(DISTINCT YEAR(call_date_ts)) year_cnt FROM v_fire;

year_cnt
24


In [ ]:
%sql
-- 9. What week of the year in 2018 had the most re calls?
SELECT WEEKOFYEAR(call_date_ts) week, COUNT(call_no) cnt FROM v_fire 
WHERE YEAR(call_date_ts) = 2018 GROUP BY WEEKOFYEAR(call_date_ts) ORDER BY 2 DESC LIMIT 1;


week,cnt
1,121


In [ ]:
%sql
-- 10. What neighborhoods in San Francisco had the worst response time in 2018?
SELECT DISTINCT neighborhoods, (unix_timestamp(response_ts) - unix_timestamp(received_ts)) AS response_time FROM v_fire 
WHERE city = "San Francisco" AND YEAR(call_date_ts) = 2018 ORDER BY 2 DESC LIMIT 3;

neighborhoods,response_time
Financial District/South Beach,24559
Tenderloin,20429
Inner Sunset,10177
